In [17]:
import cv2
import os
import numpy as np
import pandas as pd
from datetime import datetime

In [19]:
df=pd.read_csv('record/student_record.csv')
df
attendance=os.makedirs('attendance',exist_ok=True)

file_path = 'attendance/'+datetime.now().strftime('%d-%m-%Y')+'.csv'
with open(file_path, 'w') as f:
    f.write('ID,Name,DateTime\n')

In [21]:
cap=cv2.VideoCapture(0)
classifier=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
# Create recognizer
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('recognizer/trainer.yml')

df=pd.read_csv('record/student_record.csv')
#[{id:1,name:"sudhanshu"},{id:2,name:"sudhanshu"}]


while(True):
    res,frame=cap.read()
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces=classifier.detectMultiScale(gray,1.3,minNeighbors=5)
    for x,y,w,h in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
        id_,conf=recognizer.predict(gray[y:y+h,x:x+w])
    if(conf<50):
        name=(df[df['ID'] == id_]['Name']).values[0]
        with open(file_path, 'a') as f:
            f.write(f'{id_},{name},{datetime.now().strftime("%d-%m-%Y %H:%M:%S")}\n')
            break

    else:
        name="unknown"
    cv2.putText(frame,str(name),(x,y+h),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
    cv2.imshow('Face recognition system',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()